In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#@np.vectorize
def calculatePensionAndLoan(pension,studentLoan,months,m,x,y,interestPension,interestStudentLoan,alpha):
    #m monthly income
    #x fraction overall income towards pension and student loans
    #y fraction of x towards pensions
    #alpha employer contribution as percentage
    #pension total pension
    #studentLoan student loans left to pay
    
    pensionVals = np.zeros(months)
    studentLoanVals = np.zeros(months)
    paymentsToLoans = np.zeros(months)
    paymentsToPension = np.zeros(months)
    monthVals = np.arange(1,months+1,1)
    pensionVals[0] = pension
    studentLoanVals[0] = studentLoan
    paymentsToLoans[0] = 0
    paymentsToPension[0] = 0
    
    for i in range(1,months):
        
        pensionPayment = 0
        loanPayment = 0
        
        #if student loan is paid or been 30 years, put the money towards pension
        if i > 360:
            pensionPayment += m*x*(1-y)
            loanPayment = 0
            studentLoan = 0
        elif studentLoan - m*x*(1-y) < 0: #if last payment to student loan, put money towards pension
            pensionPayment += m*x*(1-y) - studentLoan
            loanPayment = studentLoan
        else: #make payment to student loan
            loanPayment = m*x*(1-y)
            
        studentLoan -= loanPayment 
        
        pensionPayment += m*x*y + m*alpha/100
        pension += pensionPayment
        
        #add interest
        pension *= interestPension
        studentLoan *= interestStudentLoan
        
        pensionVals[i] = pension
        studentLoanVals[i] = studentLoan
        
        paymentsToLoans[i] = paymentsToLoans[i-1] + loanPayment
        paymentsToPension[i] = paymentsToPension[i-1] + pensionPayment
        
    return [monthVals,pensionVals,studentLoanVals,paymentsToLoans,paymentsToPension]
    


In [32]:
def calculateFinances(
    pension,
    loan,
    investment,
    totalMonths,
    initialAnnualGrossSalary,
    finalAnnualGrossSalary,
    percentagePensionPayment,
    percentageLoanPayment,
    percentageInvestmentPayment,
    employerPercentageContribution,    
    monthlyLoanMultiplier,
    monthlyPensionMultiplier,
    monthlyInvestmentMultiplier):
    
    loans = np.zeros(totalMonths)
    pensions = np.zeros(totalMonths)
    investments = np.zeros(totalMonths)
    loanPayments = np.zeros(totalMonths)
    pensionPayments = np.zeros(totalMonths)
    investmentPayments = np.zeros(totalMonths)
    
    loans[0] = loan
    pensions[0] = pension
    investments[0] = investment
    
    for i in range(1,totalMonths):
        annualSalary = calculateLinearSalary(initialAnnualGrossSalary,finalAnnualGrossSalary,totalMonths,i)
        monthlyLoanPayment = (percentageLoanPayment/100)*(annualSalary/12)
        monthlyPensionPayment = (percentagePensionPayment/100)*(annualSalary/12)
        monthlyInvestmentPayment = (percentageInvestmentPayment/100)*(annualSalary/12)
        employerMonthlyContribution = (employerPercentageContribution/100)*(annualSalary/12)
        #print(monthlyLoanPayment)
        #print(percentageLoanPayment)
        #print(monthlyPensionPayment)
        #print(monthlyInvestmentPayment)
        #print(employerMonthlyContribution)
        
        [loan,spare] = calculateLoan(loan,monthlyLoanPayment,monthlyLoanMultiplier)
        #print(pension)
        #print(monthlyPensionPayment)
        #print(spare)
        #print(monthlyPensionMultiplier)
        #print(employerMonthlyContribution)
        pension = calculatePension(pension,monthlyPensionPayment+spare,monthlyPensionMultiplier,employerMonthlyContribution)
        #print(pension)
        investment = calculateInvestment(investment,monthlyInvestmentPayment,monthlyInvestmentMultiplier)
        #print(investment)
        #print(loan)
        
        loans[i] = loan
        pensions[i] = pension
        investments[i] = investment
        loanPayments[i] = loanPayments[i-1] + monthlyLoanPayment
        pensionPayments[i] = pensionPayments[i-1] + monthlyPensionPayment
        investmentPayments[i] = investmentPayments[i-1] + monthlyInvestmentPayment
        
    return [pensions,loans,investments,loanPayments,pensionPayments,investmentPayments]
    

In [33]:
#Calculates the minimum monthly repayment for the given annual income
def calcMinStudentRepayment(annualIncome):
    if annualIncome <= 25000:
        monthlyPayment = 0
    else:
        monthlyPayment = (annualIncome - 25000)*0.09/12
        
    return monthlyPayment

In [34]:
def calculateLoan(loan,monthlyPayment,monthlyMultiplier):
    if monthlyPayment > loan:
        loan = 0
        spare = monthlyPayment - loan
    else:
        loan -= monthlyPayment
        spare = 0
        
    loan *= monthlyMultiplier
        
    return [loan,spare]

In [35]:
def calculatePension(pension,monthlyPayment,monthlyMultiplier,employerMonthlyContribution):
    pension = (pension+monthlyPayment+employerMonthlyContribution)*monthlyMultiplier
    return pension

In [36]:
def calculateInvestment(investment,monthlyPayment,monthlyMultiplier):
    investment = (investment+monthlyPayment)*monthlyMultiplier
    return investment

In [37]:
def calculateLinearSalary(initialAnnualGrossSalary,finalAnnualGrossSalary,totalMonths,currentMonth):
    m = (finalAnnualGrossSalary-initialAnnualGrossSalary)/totalMonths
    c = initialAnnualGrossSalary
    return m*currentMonth+c

**How do the gain, pension payments, loan payments and pension vary over time?**

In [41]:
from numpy import pi, sin
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, CheckButtons, TextBox

# Set up the figure
axis_color = 'lightgoldenrodyellow'
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
# Adjust the subplots region to leave some space for the sliders and buttons
fig.subplots_adjust(right=0.65)

# Initialise Variables

monthsOfPayments = 500 #(41 years)
months = np.arange(1,monthsOfPayments+1,1)

annualPensionMultiplier = 1.03
annualLoanMultiplier = 1.063

pension = 0
loan = 60000
investment = 0
totalMonths = 500
initialAnnualGrossSalary = 40000
finalAnnualGrossSalary = 40000
percentagePensionPayment = 0.10
percentageLoanPayment = 0.01
percentageInvestmentPayment = 0
employerPercentageContribution = 5
monthlyPensionMultiplier = annualPensionMultiplier**(1/12)
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
monthlyInvestmentMultiplier = monthlyPensionMultiplier

[pensions,loans,investments,loanPayments,pensionPayments,investmentPayments] = calculateFinances(
    pension,
    loan,
    investment,
    totalMonths,
    initialAnnualGrossSalary,
    finalAnnualGrossSalary,
    percentagePensionPayment,
    percentageLoanPayment,
    percentageInvestmentPayment,
    employerPercentageContribution,
    monthlyLoanMultiplier,
    monthlyPensionMultiplier,
    monthlyInvestmentMultiplier)
    
gains = pensions - loanPayments

# Draw the initial plot
# The 'line' variable is used for modifying the line later
[linePensions] = ax.plot(months/12, pensions, linewidth=2, color='yellow',label='Pensions')
[lineLoanPayments] = ax.plot(months/12, loanPayments, linewidth=2, color='red',label='Loan Payments')
[lineLoans] = ax.plot(months/12, loans, linewidth=2, color='orange',label='Student Loan')
[linePensionPayments] = ax.plot(months/12, pensionPayments, linewidth=2, color='blue',label='Pension Payments')
[lineGains] = ax.plot(months/12, gains, linewidth=2, color='green',label='Gain')

ax.set_xlim([0, 70])
ax.set_ylim([0, 300000])
ax.set_xlabel("Years")
ax.set_ylabel("Amount £")
ax.legend(loc='upper left')

# Draw monthly loan payment slider
loanPercentage_slider_ax  = fig.add_axes([0.80, 0.80, 0.1, 0.03], facecolor=axis_color)
loanPercentage_slider = Slider(loanPercentage_slider_ax, 
                               'Monthly Loan Payments £',
                               0, 
                               30, 
                               valinit=percentageLoanPayment)

# Draw fixed percentage pension payment slider
pensionPercentage_slider_ax = fig.add_axes([0.8, 0.75, 0.10001, 0.03], facecolor=axis_color)
pensionPercentage_slider = Slider(pensionPercentage_slider_ax, 
                                  'Percentage Pension Payments %',
                                  0, 
                                  30, 
                                  valinit=percentagePensionPayment)

# Function to update slider when the textbox value is changed
def update_sliders(val):
    pensionPercentage_slider.val = float(text_box.text)
    plt.draw()
    update_plot(val)

axbox = fig.add_axes([0.80, 0.70, 0.1, 0.02], facecolor=axis_color)
text_box = TextBox(axbox, '', initial=str(round(pensionPercentage_slider.val,2)))
text_box.on_submit(update_sliders)

# Function to update textbox when the slider has changed
def updateTextbox(val):
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
    
pensionPercentage_slider.on_changed(updateTextbox)

# Draw loan interest slider
annualLoanMultiplier_slider_ax = fig.add_axes([0.8, 0.45, 0.1, 0.03], facecolor=axis_color)
annualLoanMultiplier_slider = Slider(annualLoanMultiplier_slider_ax, 
                                     'Annual Loan Multiplier £', 
                                     1, 
                                     1.25, 
                                     valinit=annualLoanMultiplier)

# Draw pension interest slider
annualPensionMultiplier_slider_ax = fig.add_axes([0.8, 0.40, 0.1, 0.03], facecolor=axis_color)
annualPensionMultiplier_slider = Slider(annualPensionMultiplier_slider_ax, 
                                        'Annual Pension Multiplier £', 
                                        1, 
                                        1.25, 
                                        valinit=annualPensionMultiplier)

# Draw initial loan slider
initialLoan_slider_ax = fig.add_axes([0.8, 0.35, 0.1, 0.03], facecolor=axis_color)
initialLoan_slider = Slider(initialLoan_slider_ax, 
                            'Initial Loan £', 
                            0, 
                            100000, 
                            valinit=loan)

# Draw initial pension slider
initialPension_slider_ax = fig.add_axes([0.8, 0.30, 0.1, 0.03], facecolor=axis_color)
initialPension_slider = Slider(initialPension_slider_ax, 
                               'Initial Pension £', 
                               0, 
                               100000, 
                               valinit=pension)

# Draw employer contribution slider
employerContribution_slider_ax = fig.add_axes([0.8, 0.25, 0.1, 0.03], facecolor=axis_color)
employerContribution_slider = Slider(employerContribution_slider_ax, 
                                     'Employer Contribution %', 
                                     0, 
                                     7, 
                                     valinit=employerPercentageContribution)

# Draw a checkbox for linear increase in salary
rax = fig.add_axes([0.8, 0.60, 0.1, 0.05],facecolor=axis_color)
labels = ["Linear Salary"]
visibility = ["True"]
check = CheckButtons(rax, labels, visibility)

# Define an action for modifying the line when any slider's value changes
def update_plot(val):
    
    pension = initialPension_slider.val
    loan = initialLoan_slider.val
    investment = 0
    totalMonths = 500
    initialAnnualGrossSalary = 40000
    finalAnnualGrossSalary = 40000
    percentagePensionPayment = pensionPercentage_slider.val
    percentageLoanPayment = loanPercentage_slider.val
    percentageInvestmentPayment = 0
    employerPercentageContribution = employerContribution_slider.val
    monthlyPensionMultiplier = annualPensionMultiplier_slider.val**(1/12)
    monthlyLoanMultiplier = annualLoanMultiplier_slider.val**(1/12)
    monthlyInvestmentMultiplier = monthlyPensionMultiplier
    
    [pensions,loans,investments,loanPayments,pensionPayments,investmentPayments] = calculateFinances(
        pension,
        loan,
        investment,
        totalMonths,
        initialAnnualGrossSalary,
        finalAnnualGrossSalary,
        percentagePensionPayment,
        percentageLoanPayment,
        percentageInvestmentPayment,
        employerPercentageContribution,
        monthlyLoanMultiplier,
        monthlyPensionMultiplier,
        monthlyInvestmentMultiplier)
    
    gains = pensions - loanPayments
    
    linePensions.set_ydata(pensions)
    lineLoanPayments.set_ydata(loanPayments)
    linePensionPayments.set_ydata(pensionPayments)
    lineLoans.set_ydata(loans)
    lineGains.set_ydata(gains)
    fig.draw()
    
loanPercentage_slider.on_changed(update_plot)
pensionPercentage_slider.on_changed(update_plot)
annualPensionMultiplier_slider.on_changed(update_plot)
annualLoanMultiplier_slider.on_changed(update_plot)
initialLoan_slider.on_changed(update_plot)
employerContribution_slider.on_changed(update_plot)
initialPension_slider.on_changed(update_plot)

# Add a button for resetting the parameters
reset_button_ax = fig.add_axes([0.8, 0.2, 0.1, 0.04])
reset_button = Button(reset_button_ax, 'Reset', color=axis_color, hovercolor='0.975')

def reset_button_on_clicked(mouse_event):
    loanPercentage_slider.reset()
    annualPensionMultiplier_slider.reset()
    annualLoanMultiplier_slider.reset()
    initialLoan_slider.reset()
    employerContribution_slider.reset()
    initialPension_slider.reset()
    pensionPercentage_slider.reset()
    
reset_button.on_clicked(reset_button_on_clicked)

plt.show()

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 94, in updateTextbox
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 820, in set_val
    self._notify_submit_observers()
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 765, in _notify_submit_observers
    func(self.text)
  File "<ipython-input-41-116bfb0463e7>", line 86, in update_sliders
    update_plot(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 re

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 94, in updateTextbox
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 820, in set_val
    self._notify_submit_observers()
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 765, in _notify_submit_observers
    func(self.text)
  File "<ipython-input-41-116bfb0463e7>", line 86, in update_sliders
    update_plot(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 re

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 94, in updateTextbox
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 820, in set_val
    self._notify_submit_observers()
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 765, in _notify_submit_observers
    func(self.text)
  File "<ipython-input-41-116bfb0463e7>", line 86, in update_sliders
    update_plot(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 re

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 94, in updateTextbox
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 820, in set_val
    self._notify_submit_observers()
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 765, in _notify_submit_observers
    func(self.text)
  File "<ipython-input-41-116bfb0463e7>", line 86, in update_sliders
    update_plot(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 re

TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 94, in updateTextbox
    text_box.set_val(str(round(pensionPercentage_slider.val,2)))
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 820, in set_val
    self._notify_submit_observers()
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 765, in _notify_submit_observers
    func(self.text)
  File "<ipython-input-41-116bfb0463e7>", line 86, in update_sliders
    update_plot(val)
  File "<ipython-input-41-116bfb0463e7>", l

...

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in up

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in up

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in up

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in up

Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in update_plot
    fig.draw()
TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'
Traceback (most recent call last):
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 417, in _update
    self.set_val(val)
  File "C:\Users\s3nt1\Anaconda3\lib\site-packages\matplotlib\widgets.py", line 438, in set_val
    func(val)
  File "<ipython-input-41-116bfb0463e7>", line 183, in up

**Explanation**

 - The gain here is the total amount of money you have (including pension payments) - the total money you have lost (loan payments).

 - Putting money towards your pension increases your gain much more than putting money towards your student loan and trying to pay it off. You can see this by varying the loan payments and comparing this to varying the pensions payments.
 - Both the interest on the loan, and the initial loan don't matter if you are making the minimum payments. The amount you pay only depends on your income. This means its more like a tax than a loan.
 - The free money from your employer contribution does have quite a big affect on your pension.

**How does the total student loan payment after 30 years vary for different incomes, assuming you only make the minimum payments?**

In [ ]:
def totalPaymentsAfter30Years(annualIncome,loan,monthlyLoanMultiplier):
    monthlyIncome = annualIncome/12
    repayment = calcMinStudentRepayment(annualIncome)
    totalPayment30Years = 0
    for j in range(12*30):
        if loan == 0:
            totalPayment30Years += 0
            break
        else:
            [loan,spare] = calculateLoan(loan,repayment,monthlyLoanMultiplier)
            totalPayment30Years += repayment-spare

    return totalPayment30Years

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

#set the constants
annualIncomes = np.arange(0,150000,1000)
annualLoanMultiplier = 1.063
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)

#calculate the initial values of the figure
initialLoan = 56000
totalPayments = np.arange(0,150000,1000)
for i,annualIncome in enumerate(annualIncomes):
    totalPayments[i] = totalPaymentsAfter30Years(annualIncome,initialLoan,monthlyLoanMultiplier)   
loans = [initialLoan for i in annualIncomes] 

#the color of the sliders
axis_color = 'lightgoldenrodyellow'

#initialise the figure
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

#get the indices of the points where the line intersects with the initial loan
idx = np.argwhere(np.diff(np.sign(totalPayments - loans))).flatten()

#create the initial line on the plot
[line] = ax.plot(annualIncomes, totalPayments, linewidth=1, color='green')
#plot the initial horizontal line for the loans
[line2] = ax.plot(annualIncomes,loans,linewidth=1,color='red')
#plot a vertical line where the lines intersect
[line3] = ax.plot((annualIncomes[idx],annualIncomes[idx]),(-10000,150000))

# Adjust the subplots region to leave some space for the sliders and buttons
fig.subplots_adjust(right=0.65)

#add labels
ax.set_xlabel("Income £")
ax.set_ylabel("Total Repayment £")
#ax.set_xticks(np.arange(min(annualIncomes), max(annualIncomes)+1, 10000))

ax.xaxis.set_major_locator(plt.MultipleLocator(10000))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2000))
ax.yaxis.set_major_locator(plt.MultipleLocator(10000))
ax.yaxis.set_minor_locator(plt.MultipleLocator(2000))

#show the figure
plt.show()

# Draw initial loan slider
initialLoan_slider_ax = fig.add_axes([0.82, 0.60, 0.1, 0.03], facecolor=axis_color)
initialLoan_slider = Slider(initialLoan_slider_ax, 'Initial Loan £', 0, 100000, valinit=initialLoan)

# Create a function to update the lines when the sliders are changed
def sliders_on_changed(val):
    
    initialLoan = initialLoan_slider.val
    for i,annualIncome in enumerate(annualIncomes):
        totalPayments[i] = totalPaymentsAfter30Years(annualIncome,initialLoan,monthlyLoanMultiplier)
    loans = [initialLoan for i in annualIncomes]

    line.set_ydata(totalPayments)
    line2.set_ydata(loans)
    idx = np.argwhere(np.diff(np.sign(totalPayments - loans))).flatten()
    line3.set_xdata(annualIncomes[idx])
    fig.canvas.draw_idle()
    
initialLoan_slider.on_changed(sliders_on_changed)

**How does varying the interest on the pension affect the final gain after 30 years? Assuming you pay the minimum student loan repayments. Annual Income = £30,000. Initial Student Loan £54,000. Monthly Pension Payments**

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 30*12 #(41 years)
yearlyIncome = 30000
monthlyIncome = yearlyIncome/12
x = 0.1
minYearlyStudentLoanPayment = (yearlyIncome - 25000)*.09
minMonthlyStudentLoanPayment = minYearlyStudentLoanPayment/12
y_max = 1-minMonthlyStudentLoanPayment/(monthlyIncome*x)
annualLoanMultiplier = 1.07
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
employerContribution = 0

start = 1.03
stop = 1.20
increment = 0.005
numVals = int((stop-start)/increment)
annualInvestmentMultipliers = np.arange(1.03, 1.20, 0.005) #every 0.5% increase in interest
gains = np.zeros(numVals+1)

for i,annualInvestmentMultiplier in enumerate(annualInvestmentMultipliers):
    monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
    [months,pensions,loans,loanPayments,pensionPayments] = calculatePensionAndLoan(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,x,y_max,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)
    gain = pensions[-1] - loanPayments[-1] - pensionPayments[-1]
    gains[i] = gain
    
gain = pensions - loanPayments - pensionPayments
plt.plot(annualInvestmentMultipliers,gains,color='g')
plt.xlabel('Annual Pension Multiplier')
plt.ylabel('Gain(£)')
plt.show()

**Question: Given a certain fraction of money x put towards student loans and pension/investment, how much of this money should be put to paying off my student loan rather than pension/investment? Calculate loads of different combinations of x and y.** <br> <br>
Initial Pension/Investment = 0 <br>
Initial Student Loan = 54000 <br>
Months of payments = 500 = 41.6 years <br>
Monthly Income (fixed) = 2500 = 30000 <br>
Pension/Investment interest monthly multiplier = 1.00407 (yearly = 1.05 i.e. 5%) <br>
Student Loan interest monthly multiplier = 1.00487 (yearly = 1.06 i.e. 6%) <br>
Employer contribution multiplier = 0 <br>

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
monthlyIncome = yearlyIncome/12
annualInvestmentMultiplier = 1.03
annualLoanMultiplier = 1.07
monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
employerContribution = 0

In [ ]:
@np.vectorize
def finalGain(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,x,y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution):
    [months,pensions,loans,loanPayments,pensionPayments] = calculatePensionAndLoan(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,x,y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)
    gain = pensions - loanPayments - pensionPayments
    return gain[-1]

In [ ]:
#Create a plot of loads of different values of x and y and the final gain of money

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

%matplotlib notebook

# Plot a wireframe of the Gain in money from putting money away against the fraction of total income x put away and the 
# fraction y of this (e.g. m*x*y) put towards the pension. m*x*(1-y) is put towards student loans.
x = np.arange(0, 0.405, 0.005)
y = np.arange(0, 1.005, 0.005)
X, Y = np.meshgrid(x, y)

#######################################################################
#Only some combinations of x and y are possible. You cannot pay nothing towards student loans e.g. y =! 1.

#Only allow points for which the loan payment is at least 9% of total income
#idx = X*(1-Y) >= 0.09
#X[~idx] = np.nan
#Y[~idx] = np.nan

#Only allow points for which the loan payment is at least 9% of income above 25k
m = 2500
idx = m*X*(1-Y) >= (m-25000/12)*0.09
X[~idx] = np.nan
Y[~idx] = np.nan

#####################################################################

In [ ]:
#Set up the figure
fig = plt.figure()
ax = fig.gca(projection='3d')

#Calculate the final gain
R = finalGain(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,X,Y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)

#Scale the axes
Z = R/1000.0

# Plot a wireframe
ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)

# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%i'))

ax.set_xlabel('x Fraction of total income', fontsize=10)
ax.set_ylabel('y Pension/Investment Fraction',fontsize = 10)
ax.set_zlabel('z Gain in £1000s')

plt.show()

Explanation Of Wireframe:

Plotted on x axis is the fraction of total income put towards paying both student loans and putting away in investments/pension. y is the fraction of this money put towards investment/pensions, 1-y is the money put towards student loans. On the z axis is the total gain in money in thousands of pounds, after making payments for just over 40 years.

The graph clearly shows that for every value of x, putting as much money as possible towards investments/pension is better than paying off student loans. The sudden change in the curvature of the graph as y becomes higher is because  

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
monthlyIncome = yearlyIncome/12
x = 0.10
#y = 0.91
#m*x*(1-y) = loanPayment
#minYearlyStudentLoanPayment = (yearlyIncome - 25000)*.09
#minMonthlyStudentLoanPayment = minYearlyStudentLoanPayment/12
#y_max = 1-minMonthlyStudentLoanPayment/(monthlyIncome*x)

annualInvestmentMultiplier = 1.03
annualLoanMultiplier = 1.07
monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)

employerContribution = 0

ys = np.arange(0.1,0.9,0.1)

for y in ys:

    [months,pensions,loans,loanPayments,pensionPayments] = calculatePensionAndLoan(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,x,y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)

    gain = pensions - loanPayments - pensionPayments
    plt.figure()
    plt.plot(months/12,pensionPayments,color='g', label='Pension Payments')
    plt.plot(months/12,loanPayments,color='r', label='Loan Payments')
    plt.plot(months/12,gain,color='b', label='Gain')
    plt.plot(months/12,pensions,color='y',label='Pension/Investment')
    plt.plot(months/12,loans,color='orange',label='Student Loan')
    plt.legend(loc='upper left')
    plt.xlabel('Years')
    plt.ylabel('Amount(£)')
    plt.show()

**Question: What does this wireframe look like for various incomes?**

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 500 #(41 years)
#yearlyIncome = 30000
#monthlyIncome = yearlyIncome/12
annualInvestmentMultiplier = 1.03
annualLoanMultiplier = 1.07
monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
employerContribution = 0

yearlyIncomes = np.arange(25000,60000,5000) #from 25k to 60k in 5k increments

for yearlyIncome in yearlyIncomes:

    monthlyIncome = yearlyIncome/12

    #Set up the figure
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    #Calculate the final gain
    R = finalGain(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,X,Y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)

    #Scale the axes
    Z = R/1000.0

    # Plot a wireframe
    ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)

    # Customize the z axis.
    #ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%i'))

    ax.set_xlabel('x Fraction of total income', fontsize=10)
    ax.set_ylabel('y Pension/Investment Fraction',fontsize = 10)
    ax.set_zlabel('z Gain in £1000s')

    plt.show()

**Answer: No matter your income or the fraction x of this that you pay, always pay the minimum amount possible towards your student loan and put as much money as possible towards investment/pension, even if the interest on that is low.**

**Question: What does this wireframe look like for various investment/pension interest rates?**

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
monthlyIncome = yearlyIncome/12
#annualInvestmentMultiplier = 1.03
#monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
annualLoanMultiplier = 1.07
monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
employerContribution = 0

annualInvestmentMultipliers = np.arange(1.03,1.12,0.01) #from 3% to 12% in 1% increments

for annualInvestmentMultiplier in annualInvestmentMultipliers:

    monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)

    #Set up the figure
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    #Calculate the final gain
    R = finalGain(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,X,Y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)

    #Scale the axes
    Z = R/1000.0

    # Plot a wireframe
    ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)

    # Customize the z axis.
    #ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%i'))

    ax.set_xlabel('x Fraction of total income', fontsize=10)
    ax.set_ylabel('y Pension/Investment Fraction',fontsize = 10)
    ax.set_zlabel('z Gain in £1000s')

    plt.show()

**Answer: The bigger the investment interest the better. Same thing applies, pay the minimum student loan payments and put all of your money towards investments/pension.**

**Question: What does this wireframe look like for various student loan interest rates?**

In [ ]:
initialInvestment = 0
initialStudentLoan = 54000
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
monthlyIncome = yearlyIncome/12
annualInvestmentMultiplier = 1.03
monthlyInvestmentMultiplier = annualInvestmentMultiplier**(1/12)
#annualLoanMultiplier = 1.07
#monthlyLoanMultiplier = annualLoanMultiplier**(1/12)
employerContribution = 0

annualLoanMultipliers = np.arange(1.03,1.07,0.01) #from 3% to 7% in 1% increments

for annualLoanMultiplier in annualLoanMultipliers:

    monthlyLoanMultiplier = annualLoanMultiplier**(1/12)

    #Set up the figure
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    #Calculate the final gain
    R = finalGain(initialInvestment,initialStudentLoan,monthsOfPayments,monthlyIncome,X,Y,monthlyInvestmentMultiplier,monthlyLoanMultiplier,employerContribution)

    #Scale the axes
    Z = R/1000.0

    # Plot a wireframe
    ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)

    # Customize the z axis.
    #ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%i'))

    ax.set_xlabel('x Fraction of total income', fontsize=10)
    ax.set_ylabel('y Pension/Investment Fraction',fontsize = 10)
    ax.set_zlabel('z Gain in £1000s')

    plt.show()

**Answer: The student loan interest does not matter because you can just pay the minimum amount, the minimum amount you pay only depends on your income. Paying student loan payments is essentially just like paying a higher tax on your income.**

**Question: For a given x, what percentage of money should you put towards investment/pensions rather than student loans for maximum gain**

In [ ]:
#print(Z)
#print(Z[:,-1])

#Find the maximum possible gain for each value of x
maxGains = np.zeros(len(Z[0,:]))
#print(len(maxGains))
for i in range(0,len(Z[0,:])):
    #idx = Z[:,i] == max(Z[:,i])
    #print(max(Z[:,i]))
    maxGains[i] = max(Z[:,i])

#print(maxGains)
#print(len(maxGains))
xs = X[50,:]
ys = Y[50,:]
    
#Plot the maximum possible gain against x
%matplotlib notebook
plt.plot(xs,maxGains)
plt.xlabel('x Fraction of total Income')
plt.ylabel('zMax Maximum possible gain £1000s')
plt.show()

#Ybest = Y[idx]
#print(X)
#print(Y)
#print(Ybest)

Plot Explanation:

The final gain you get from your investments/pension is directly proportional to the amount of money you put towards investments. If you double the money you put away each month, then you'll end up with double the gain at the end. 

In [ ]:
#Choose a value of x and find the max gain for this and the corresponding y value
#xChoice = 0.33
#gainChoice = maxGains[x == xChoice][0] #the best gain
#yBooleanArray = Z[:,x == xChoice] == max(Z[:,x == xChoice])
#yIdx = np.where(yBooleanArray == [True])[0][0]
#yChoice = y[yIdx] #the best y value
#print('The maximum possible gain for x = '+str(xChoice)+' is: '+str(gainChoice))
#print('The corresponding optimal y value is: '+str(yChoice))
#print('The percentage of income to pay to pension is: '+str(yChoice*xChoice*100))
#print('The percentage of income to pay to student Loans is: '+str((1-yChoice)*xChoice*100))

#For each value of x find the maximum gain and corresponding y value
loanPercentages = np.zeros(len(xs))

#print(x)
for i,x in enumerate(xs):

    if not np.isnan(x):
        #print(xChoice)
        #print(maxGains)
        gainChoice = maxGains[xs == x]
        #print(gainChoice)
        print(x)
        #print(Z[:,xs == x])
        #print(max(Z[:,xs == x]))
        yBooleanArray = Z[:,xs == x] == max(Z[:,xs == x])
        #print(yBooleanArray)
        yIdx = np.where(yBooleanArray == [True])[0][0]
        print(yIdx)
        y = ys[yIdx] #the best y value
        print(y)
        loanPercentages[i] = (1-y)*x*100


#Plot the percentages
%matplotlib notebook
plt.plot(xs,loanPercentages)
plt.xlabel('x Fraction of total Income')
plt.ylabel('Percentage To Student Loan For Maximum Gain')
plt.show()

**Question: Does adding interest every month vs every year make a difference?**

Annual interest rate $i_r = 0.10$, $i = 1.10$ <br>
Monthly interest rate $x$ is calculated from $1.10=x^{12}$ so $x=10^{log(1.10)/12}=1.00797414$ <br>
Initial amount = 1000 <br>
Yearly: $1000 \times 1.10 = 1100$ <br>
Monthly: $1000 \times 1.00797^{12} = 1100$ <br>

So what happens if you make payments of 100 every month instead of paying a lump sum of 1200 yearly? Assume interest is added after each payment.

Yearly: $(1000+1200) \times 1.10 = 2420$ <br>
Monthly: $((((1000+100)\times1.00797)+100)\times1.00797)... = 2364$ <br>
        $= 1000 \times 1.00797^{12} + 100\times(1.00797^{12}+1.00797^{11}+...+1.00797)$

**Answer: Having a yearly interest is better than having a monthly interest. This is because the last payment of 100 only gains an interest of 1.00797 and only the first payment of 100 gains the full 1.10 interest.**

Note paying all of the 1200 in the first month is no different than to having an annual interest rate <br>
$(1000+1200)\times1.00797^{12}=(1000+1200)\times1.10$

**Question: Is it worth having any money in a pension or should I just put it all towards an investment? Considering the tax free benefits and employer contribution.**

In [ ]:
#Need to know interest rates
#Login details for standard life

def calculateInterest(initialAmount,monthlyContribution,annualInterestMultiplier,months):
    monthlyInterestMultiplier = annualInterestMultiplier**(1/12)
    amount = initialAmount
    amounts = np.zeros(months)
    
    for i in range(months):
        amount = (amount+monthlyContribution)*monthlyInterestMultiplier
        amounts[i] = amount
        
    return amounts

In [ ]:
initialPension = 520
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
employerMonthlyContribution = 0.05*yearlyIncome/12
print(employerMonthlyContribution)
monthlyContribution = 0.04*yearlyIncome/12
print(monthlyContribution)
annualPensionMultiplier = 1.03

months = np.arange(1,monthsOfPayments+1,1)

pensions = calculateInterest(initialPension,monthlyContribution+employerMonthlyContribution,annualPensionMultiplier,monthsOfPayments)
pensionGains = pensions - monthlyContribution*months

pensions2 = calculateInterest(initialPension,monthlyContribution+employerMonthlyContribution,annualPensionMultiplier+0.02,monthsOfPayments)
pensionGains2 = pensions2 - monthlyContribution*months

investments = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier,monthsOfPayments)
investmentGains = investments - monthlyContribution*months

investments2 = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier+0.02,monthsOfPayments)
investmentGains2 = investments2 - monthlyContribution*months

investments3 = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier+0.04,monthsOfPayments)
investmentGains3 = investments3 - monthlyContribution*months

investments4 = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier+0.06,monthsOfPayments)
investmentGains4 = investments4 - monthlyContribution*months

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,pensionGains,label='Pension With EC, Interest 3%')
plt.plot(months/12,pensionGains2,label='Pension With EC, Interest 5%')
plt.plot(months/12,investmentGains,label='Investment Without EC, Interest 3%')
plt.plot(months/12,investmentGains2,label='Investment Without EC, Interest 5%')
plt.plot(months/12,investmentGains3,label='Investment Without EC, Interest 7%')
plt.plot(months/12,investmentGains4,label='Investment Without EC, Interest 9%')
plt.xlabel('Years')
plt.ylabel('Gain From Pension')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

In [ ]:
initialPension = 520
monthsOfPayments = 500 #(41 years)
yearlyIncome = 30000
employerMonthlyContribution = 0.05*yearlyIncome/12
#print(employerMonthlyContribution)
monthlyContribution = 200
#print(monthlyContribution)
annualPensionMultiplier = 1.03

months = np.arange(1,monthsOfPayments+1,1)

pensions = calculateInterest(initialPension,monthlyContribution+employerMonthlyContribution,annualPensionMultiplier,monthsOfPayments)
pensionGains = pensions - monthlyContribution*months

investments = calculateInterest(initialPension,monthlyContribution-monthlyContribution*0.2,annualPensionMultiplier,monthsOfPayments)
investmentGains = investments - monthlyContribution*months

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,pensionGains,label='Pension With EC, Interest 3%, 20% Tax Relief')
plt.plot(months/12,investmentGains,label='Investment Without EC, Interest 3%')
plt.xlabel('Years')
plt.ylabel('Gain From Pension')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

**Income After Income Tax Against Gross Income**

In [ ]:
def calculatePersonalAllowance(grossAnnualIncome):
    if (grossAnnualIncome <= 100000):
        personalAllowance = 11850
    elif (grossAnnualIncome >= 123700):
        personalAllowance = 0
    elif (100000 <= grossAnnualIncome <= 123700):
        personalAllowance = -0.5*grossAnnualIncome+61850
    return personalAllowance

In [ ]:
def applyIncomeTax(grossIncome):
    personalAllowance = calculatePersonalAllowance(grossIncome)
    originalIncome = grossIncome
    taxedAmount = 0
    
    if grossIncome >= 150000:
        taxedAmount += (grossIncome-150000)*0.45
        grossIncome = 150000
    if grossIncome >= 46351:
        taxedAmount += (grossIncome-46351)*0.40
        grossIncome = 46351
    if grossIncome >= personalAllowance:
        taxedAmount += (grossIncome-personalAllowance)*0.20
        
    return originalIncome-taxedAmount

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

grossIncomes = np.arange(0,100000,1000)
taxedIncomes = np.zeros(100)
for i,grossIncome in enumerate(grossIncomes):
    taxedIncome = applyIncomeTax(grossIncome)
    taxedIncomes[i] = taxedIncome

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

ax.plot(grossIncomes,taxedIncomes)
ax.set_ylabel('Income After Income Tax')
ax.set_xlabel('Gross Income')
ax.xaxis.set_major_locator(plt.MultipleLocator(10000))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2000))
ax.yaxis.set_major_locator(plt.MultipleLocator(10000))
ax.yaxis.set_minor_locator(plt.MultipleLocator(2000))
plt.show()

**How does capital gains tax affect the gains from your investments?**

In [ ]:
def applyCapitalGainsTax(grossAnnualIncome,taxableGain,includeCapitalTaxGainAllowance=False):
    #taxableGain Any gain from investments that could be taxed (Not in an ISA)
    #includeCapitalTaxGainAllowance Boolean for whether or not to include the 11700 allowance or not
    if includeCapitalTaxGainAllowance:
        capitalTaxGainsAllowance = 11700
    else:
        capitalTaxGainsAllowance = 0
    taxableGain = taxableGain - capitalTaxGainsAllowance    
    
    personalIncomeAllowance = calculatePersonalAllowance(grossAnnualIncome)
    taxableIncome = grossAnnualIncome - personalIncomeAllowance
    
    taxRateQuantifier = taxableIncome + taxableGain
    
    finalGain = 0
    if (taxRateQuantifier >= 46350):
        finalGain += 0.8*taxableGain*((taxRateQuantifier-46350)/taxRateQuantifier)
        finalGain += 0.9*taxableGain*(1-((taxRateQuantifier-46350)/taxRateQuantifier))
    else:
        finalGain += 0.9*taxableGain
    
    return finalGain

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

grossAnnualIncome = 40000
taxableGains = np.arange(0,100000,1000)
finalGains = np.arange(0,100000,1000)
for i,taxableGain in enumerate(taxableGains):
    finalGains[i] = applyCapitalGainsTax(grossAnnualIncome,taxableGain,False)
    
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

ax.plot(taxableGains,finalGains)
ax.set_xlabel('Taxable Gains £')
ax.set_ylabel('Final Gains After CGT £')
ax.xaxis.set_major_locator(plt.MultipleLocator(10000))
ax.xaxis.set_minor_locator(plt.MultipleLocator(2000))
ax.yaxis.set_major_locator(plt.MultipleLocator(10000))
ax.yaxis.set_minor_locator(plt.MultipleLocator(2000))
plt.show()

How a small amount a month can grow through interest

In [ ]:
initialPension = 0
monthsOfPayments = 500 #(41 years)
monthlyContribution = 5
annualPensionMultiplier = 1.03

months = np.arange(1,monthsOfPayments+1,1)

investments = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier,monthsOfPayments)
investmentGains = investments - monthlyContribution*months

investments2 = calculateInterest(initialPension,10,annualPensionMultiplier,monthsOfPayments)
investmentGains2 = investments2 - monthlyContribution*months

investments3 = calculateInterest(initialPension,15,annualPensionMultiplier,monthsOfPayments)
investmentGains3 = investments3 - monthlyContribution*months

investments4 = calculateInterest(initialPension,20,annualPensionMultiplier,monthsOfPayments)
investmentGains4 = investments4 - monthlyContribution*months

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,investmentGains,label='Investment £5/Month, Interest 3%')
plt.plot(months/12,investmentGains2,label='Investment £10/Month, Interest 3%')
plt.plot(months/12,investmentGains3,label='Investment £15/Month, Interest 3%')
plt.plot(months/12,investmentGains4,label='Investment £20/Month, Interest 3%')
plt.xlabel('Years')
plt.ylabel('Gain')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

After 30 years of paying £5 a month you can GAIN £1100 <br>
After 30 years of paying £10 a month you can GAIN £4000 <br>
After 30 years of paying £15 a month you can GAIN £6900 <br>
After 30 years of paying £20 a month you can GAIN £9800 <br>

Doubling your monthly contribution more than doubles your final gain

How does the gain vary with monthly payment?

In [ ]:
initialPension = 0
monthsOfPayments = 500 #(41 years)
monthlyContributions = np.arange(0,300,50)
annualPensionMultiplier = 1.03

months = np.arange(1,monthsOfPayments+1,1)

for monthlyContribution in monthlyContributions:
    investments = calculateInterest(initialPension,monthlyContribution,annualPensionMultiplier,monthsOfPayments)
    investmentGains = investments - monthlyContribution*months
    

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,investmentGains,label='Investment £5/Month, Interest 3%')
plt.plot(months/12,investmentGains2,label='Investment £10/Month, Interest 3%')
plt.plot(months/12,investmentGains3,label='Investment £15/Month, Interest 3%')
plt.plot(months/12,investmentGains4,label='Investment £20/Month, Interest 3%')
plt.xlabel('Years')
plt.ylabel('Gain')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

In [ ]:
initialPension = 0
monthsOfPayments = 500 #(41 years)
monthlyContribution = 5
annualPensionMultiplier = 1.03

months = np.arange(1,monthsOfPayments+1,1)

investments = calculateInterest(initialPension,5,1.03,monthsOfPayments)
investmentGains = investments - monthlyContribution*months

investments2 = calculateInterest(initialPension,5,1.05,monthsOfPayments)
investmentGains2 = investments2 - monthlyContribution*months

investments3 = calculateInterest(initialPension,5,1.07,monthsOfPayments)
investmentGains3 = investments3 - monthlyContribution*months

investments4 = calculateInterest(initialPension,5,1.09,monthsOfPayments)
investmentGains4 = investments4 - monthlyContribution*months

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,investmentGains,label='Investment £5/Month, Annual Interest 3%')
plt.plot(months/12,investmentGains2,label='Investment £5/Month, Annual Interest 5%')
plt.plot(months/12,investmentGains3,label='Investment £5/Month, Annual Interest 7%')
plt.plot(months/12,investmentGains4,label='Investment £5/Month, Annual Interest 9%')
plt.xlabel('Years')
plt.ylabel('Gain')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

After 30 years of paying £5 a month at 3% Annual Interest you can GAIN £1100 <br>
After 30 years of paying £5 a month at 5% Annual Interest you can GAIN £2300 <br>
After 30 years of paying £5 a month at 7% Annual Interest you can GAIN £4100 <br>
After 30 years of paying £5 a month at 9% Annual Interest you can GAIN £6800 <br>

Increasing annual interest from 3% to 5% you gain an extra £1200
Increasing annual interest from 5% to 7% you gain an extra £1800
Increasing annual interest from 7% to 9% you gain an extra £2700

Increasing the interest by a fixed amount e.g. 2% has a bigger effect the larger the interest currently is.

How long will it take for me to get to a point where the interest on my investments is enough to pay for my regular outgoings?

In [ ]:
def calculateInterest2(initialAmount,monthlyContribution,annualInterestMultiplier,months):
    monthlyInterestMultiplier = annualInterestMultiplier**(1/12)
    amount = initialAmount
    amounts = np.zeros(months)
    
    for i in range(months):
        #If interest greater than regular outgoings, then retire and use interest to pay for regular outgoings
        print((amount+monthlyContribution)*(monthlyInterestMultiplier-1))
        if ((amount+monthlyContribution)*(monthlyInterestMultiplier-1) >= 1000):
            amount = amount + monthlyContribution
        #Else keep adding interest
        else:
            amount = (amount+monthlyContribution)*(1+monthlyInterestMultiplier-1)
            
        amounts[i] = amount
        
    return amounts

In [ ]:
initialPension = 0
monthsOfPayments = 500 #(41 years)
monthlyContribution = 200
annualPensionMultiplier = 1.10

months = np.arange(1,monthsOfPayments+1,1)

investments = calculateInterest2(initialPension,monthlyContribution,1.10,monthsOfPayments)
investmentGains = investments - monthlyContribution*months

investments2 = calculateInterest2(initialPension,monthlyContribution,1.07,monthsOfPayments)
investmentGains2 = investments2 - monthlyContribution*months

investments3 = calculateInterest2(initialPension,monthlyContribution,1.05,monthsOfPayments)
investmentGains3 = investments3 - monthlyContribution*months

investments4 = calculateInterest2(initialPension,monthlyContribution,1.03,monthsOfPayments)
investmentGains4 = investments4 - monthlyContribution*months

#Plot the gain from pension
%matplotlib notebook
plt.plot(months/12,investmentGains,label='Investment £200/Month, Annual Interest 10%')
plt.plot(months/12,investmentGains2,label='Investment £200/Month, Annual Interest 7%')
plt.plot(months/12,investmentGains3,label='Investment £200/Month, Annual Interest 5%')
plt.plot(months/12,investmentGains4,label='Investment £200/Month, Annual Interest 3%')
plt.xlabel('Years')
plt.ylabel('Gain')
plt.legend(loc='upper left')
plt.show()
#Account for tax relief, add on monthlyContribution*.2+NI?

In [ ]:
Need to account for the fact that when taking out money from a stock to pay for living costs, I only take out what I need so the rest of the money continues to grow still and so the money should last several years more than initially thought.

In [ ]:
Look some more at how annual and monthly interest varies for different values of interest